<a href="https://colab.research.google.com/github/SnehaRachel/Basic_Sentiment_Analysis_with_scikit-learn/blob/main/Basic_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyDrive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1Ornb_uYa-wBtTc-At9VPCVBEEJJP8hr9"}) 
downloaded.GetContentFile('train.csv')

In [ ]:
downloaded = drive.CreateFile({'id':"18mjZ1kW3ApvqAmtiKeR6t2ndJ4pg5OpT"}) 
downloaded.GetContentFile('test.csv')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics import classification_report


In [ ]:
data_train = pd.read_csv(r'train.csv', encoding='unicode_escape')


In [ ]:
data_train.head()

,ï»¿text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [ ]:
data_test = pd.read_csv(r'test.csv', encoding='unicode_escape')


In [ ]:
data_test.tail()

,ï»¿text,label
4995,This is the kind of picture John Lassiter woul...,1
4996,A MUST SEE! I saw WHIPPED at a press screening...,1
4997,NBC should be ashamed. I wouldn't allow my chi...,0
4998,This movie is a clumsy mishmash of various gho...,0
4999,Formula movie about the illegitimate son of a ...,0


In [ ]:
data_train.rename(columns = {'ï»¿text':'Movie_Review'}, inplace = True)
data_train.head()

,Movie_Review,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [ ]:
data_test.rename(columns = {'ï»¿text':'Movie_Review'}, inplace = True)
data_test.tail()

,Movie_Review,label
4995,This is the kind of picture John Lassiter woul...,1
4996,A MUST SEE! I saw WHIPPED at a press screening...,1
4997,NBC should be ashamed. I wouldn't allow my chi...,0
4998,This movie is a clumsy mishmash of various gho...,0
4999,Formula movie about the illegitimate son of a ...,0


In [ ]:
data_test.head()

,Movie_Review,label
0,I always wrote this series off as being a comp...,0
1,1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...,0
2,This movie was so poorly written and directed ...,0
3,The most interesting thing about Miryang (Secr...,1
4,"when i first read about ""berlin am meer"" i did...",0


In [ ]:
#Concatenating train and test datasets into one dataframe
data = pd.concat([data_train, data_test], axis=0, ignore_index=True)

In [ ]:
data

,Movie_Review,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
44995,This is the kind of picture John Lassiter woul...,1
44996,A MUST SEE! I saw WHIPPED at a press screening...,1
44997,NBC should be ashamed. I wouldn't allow my chi...,0
44998,This movie is a clumsy mishmash of various gho...,0


In [ ]:
data.iloc[44999]

Movie_Review    Formula movie about the illegitimate son of a ...
label                                                           0
Name: 44999, dtype: object

In [ ]:
for col in data_train.columns:
    print(col)

Movie_Review
label


In [ ]:
#Renaming column name for readability
data.rename(columns = {'ï»¿text':'Movie_Review'}, inplace = True)
data.head()

,Movie_Review,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [ ]:
data.tail()

,Movie_Review,label
44995,This is the kind of picture John Lassiter woul...,1
44996,A MUST SEE! I saw WHIPPED at a press screening...,1
44997,NBC should be ashamed. I wouldn't allow my chi...,0
44998,This movie is a clumsy mishmash of various gho...,0
44999,Formula movie about the illegitimate son of a ...,0


In [ ]:
data.shape

(45000, 2)

In [ ]:
#nltk required downloads
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Function to preprocess the data and return the clean copus
def get_corpus(df):
 corpus = []
 for i in range(0, 45000):
  #Word tokenization
  tokenizer = RegexpTokenizer(r'\w+')
  review=tokenizer.tokenize(data['Movie_Review'][i])
  #Lowercase conversion
  review =[word.lower() for word in review if word.isalpha()]

  # Instantiating WordNeLemmatizer class
  lemmatizer=WordNetLemmatizer()
 
  #Lemmatization
  review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords.words('english'))] 

  review = ' '.join(review) 
  corpus.append(review)
  
 return corpus

In [ ]:
corpus=get_corpus(data)

In [ ]:
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()

y = data.iloc[:, 1].values

In [ ]:
X.shape


(45000, 1500)

In [ ]:
#Splitting the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
y.shape

(45000,)

In [ ]:
y

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
#Naive Bayes Classifier
classifier = GaussianNB()
classifier.fit(X_train, y_train)


GaussianNB()

In [ ]:
y_pred_nb = classifier.predict(X_test)
y_pred_nb

array([1, 0, 0, ..., 0, 0, 1])

In [ ]:
from sklearn.metrics import accuracy_score


In [ ]:
cm_nb = accuracy_score(y_test, y_pred_nb) 
cm_nb

0.7826666666666666

In [ ]:
cm = confusion_matrix(y_test, y_pred_nb) 
cm

array([[3884,  643],
       [1313, 3160]])

In [ ]:
print(classification_report(y_test, y_pred_nb))

              precision    recall  f1-score   support

           0       0.75      0.86      0.80      4527
           1       0.83      0.71      0.76      4473

    accuracy                           0.78      9000
   macro avg       0.79      0.78      0.78      9000
weighted avg       0.79      0.78      0.78      9000



In [ ]:
#Random Forest Classifier
classifier = RandomForestClassifier(n_estimators = 300, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=300, random_state=0)

In [ ]:
y_pred_rf = classifier.predict(X_test)
y_pred_rf

array([1, 0, 1, ..., 0, 0, 1])

In [ ]:
cm_rf = confusion_matrix(y_test, y_pred_rf)
cm_rf

array([[3812,  715],
       [ 666, 3807]])

In [ ]:
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.85      0.84      0.85      4527
           1       0.84      0.85      0.85      4473

    accuracy                           0.85      9000
   macro avg       0.85      0.85      0.85      9000
weighted avg       0.85      0.85      0.85      9000



In [ ]:
as_rf = accuracy_score(y_test, y_pred_rf) 
as_rf

0.8465555555555555

In [ ]:
#Support Vector Machine
from sklearn.svm import SVC  
clf = SVC(kernel='linear') 

In [ ]:
clf.fit(X_train, y_train)

SVC(kernel='linear')

In [ ]:
y_pred_svm = classifier.predict(X_test)
y_pred_svm

array([1, 0, 1, ..., 0, 0, 1])

In [ ]:
cm_svm = confusion_matrix(y_test, y_pred_svm)
cm_svm

array([[3812,  715],
       [ 666, 3807]])

In [ ]:
print(classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

           0       0.85      0.84      0.85      4527
           1       0.84      0.85      0.85      4473

    accuracy                           0.85      9000
   macro avg       0.85      0.85      0.85      9000
weighted avg       0.85      0.85      0.85      9000



In [ ]:
as_svm = accuracy_score(y_test, y_pred_svm) 
as_svm

0.8465555555555555